# 1. Авторизация

In [3]:
# https://developers.google.com/drive/api/v3/quickstart/python?hl=ru#prerequisites
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'account_file.json'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [4]:
from googleapiclient.discovery import build

service = build('drive', 'v3', credentials=credentials)

# 2. Скачать все текущие изображения

In [11]:
def get_files(service):
    files = []
    
    results = service.files().list(
        pageSize=1000, 
        fields="nextPageToken, files(id, name, mimeType)",
    ).execute()
    nextPageToken = results.get('nextPageToken')
    
    for file in results['files']:
        if file['mimeType'] == 'image/jpeg': # только изображения
            files.append(file)
            
    while nextPageToken:
        nextPage = service.files().list(
            pageSize=1000, 
            fields="nextPageToken, files(id, name, mimeType)", 
            pageToken=nextPageToken
        ).execute()
        nextPageToken = nextPage.get('nextPageToken')
        
        for file in nextPage['files']:
            if file['mimeType'] == 'image/jpeg':
                files.append(file)
    return files

In [12]:
files = get_files(service)

In [13]:
files[0]

{'id': '1i0TZMCEbiS0O8U0yz-EFL-gzNO8mCJ6n',
 'name': 'post-6-1543763308 - Игорь Жирнов.jpg',
 'mimeType': 'image/jpeg'}

# 3. Изменить размер файла

In [17]:
from PIL import Image
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io


def get_new_size(w, h):
    n = 2000
    if max(w, h) < 2000:
        return w, h
    if w > h:
        return n, n * h // w
    else:
        return n * w // h, n

def resize_image(service, name: str, id: str, **kwargs):
    # download file
    request = service.files().get_media(fileId=id)
    fh = io.FileIO(name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print('Download %s' % name)
      
    # resize
    im = Image.open(name)
    print(im.size)
    small_im = im.resize(get_new_size(*im.size))
    print(small_im.size)
    
    # save small file
    small_name = 'small.jpg'
    small_fh = io.FileIO(small_name, 'wb')
    small_im.save(small_fh)
    media = MediaFileUpload(small_name, mimetype='image/jpeg')
    service.files().update(fileId=id, media_body=media).execute()
    
    # delete old revisions
    results = service.revisions().list(fileId=id).execute()
    for revision in sorted(results['revisions'], key=lambda x: x['modifiedTime'])[:-1]:
        revision_id = revision['id']
        service.revisions().delete(revisionId=revision_id, fileId=id).execute()

In [19]:
for file in files:
    resize_image(service, **file)
    break

Download post-6-1543763308 - Игорь Жирнов.jpg
(1280, 1011)
(1280, 1011)
